In [1]:
%matplotlib inline

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [249]:
# The code was removed by Watson Studio for sharing.

In [362]:
# The code was removed by Watson Studio for sharing.

In [363]:
body = client_d332a453df2c43ee8d5e3e2e36138336.get_object(Bucket='finalcapstone-donotdelete-pr-m3hhu5wn2z9spw',Key='calpop.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# import population data
calpop = pd.read_csv(body)
#calpop.rename(columns={'City':'name'}, inplace=True)
calpop.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010
0,San Bernardino,Adelanto,1970,2164.0,8517,18130,31765
1,Los Angeles,Agoura Hills,1982,NaN,20390,20537,20330
2,Alameda,Alameda,1854,63852.0,76459,72259,73812
3,Alameda,Albany,1908,15130.0,16327,16444,18539
4,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089


In [251]:
body = client_d332a453df2c43ee8d5e3e2e36138336.get_object(Bucket='finalcapstone-donotdelete-pr-m3hhu5wn2z9spw',Key='calloc.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# import geo coordinates
calloc = pd.read_csv(body)
calloc.rename(columns={'Name':'City'}, inplace=True)
calloc.head()

,City,Latitude,Longitude
0,Adelanto,34.582769,-117.409214
1,Agoura Hills,34.153339,-118.761675
2,Alameda,37.765206,-122.241636
3,Albany,37.886869,-122.297747
4,Alhambra,34.095286,-118.127014


In [252]:
# merge popoulation and coordinates by city
df1=calpop.merge(calloc)
df1.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010,Latitude,Longitude
0,San Bernardino,Adelanto,1970,2164.0,8517,18130,31765,34.582769,-117.409214
1,Los Angeles,Agoura Hills,1982,NaN,20390,20537,20330,34.153339,-118.761675
2,Alameda,Alameda,1854,63852.0,76459,72259,73812,37.765206,-122.241636
3,Alameda,Albany,1908,15130.0,16327,16444,18539,37.886869,-122.297747
4,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089,34.095286,-118.127014


In [253]:
# df1.groupby('County')['City'].count()

In [254]:
# filter San Diego data
df2=df1[df1['County']=='San Diego'].reset_index(drop=True)
df2.head()
#df2.shape

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010,Latitude,Longitude
0,San Diego,Carlsbad,1952,35490.0,63126,78247,105328,33.158092,-117.350594
1,San Diego,Chula Vista,1911,35490.0,63126,173556,243916,32.640053,-117.084197
2,San Diego,Coronado,1890,18790.0,26540,24100,18912,32.685886,-117.183089
3,San Diego,Del Mar,1959,5017.0,4860,4389,4161,32.959489,-117.265314
4,San Diego,El Cajon,1912,73892.0,88693,94869,99478,32.794772,-116.962528


In [280]:
# delete unwanted columns
df3=df2.drop(['County','Incorportation_date','pop_april_1980','pop_april_1990','pop_april_2000'],axis=1)
df3

,City,pop_april_2010,Latitude,Longitude
0,Carlsbad,105328,33.158092,-117.350594
1,Chula Vista,243916,32.640053,-117.084197
2,Coronado,18912,32.685886,-117.183089
3,Del Mar,4161,32.959489,-117.265314
4,El Cajon,99478,32.794772,-116.962528
5,Encinitas,59518,33.036986,-117.291983
6,Escondido,143911,33.119208,-117.086422
7,Imperial Beach,26324,32.583944,-117.113086
8,La Mesa,57065,32.767828,-117.023083
9,Lemon Grove,25320,32.742553,-117.031417


In [256]:
# create map of San Diego using latitude and longitude values
latitude = 32.7157
longitude = -117.1611
#print('The geograpical coordinate of San Diego are {}, {}.'.format(latitude, longitude))

map1 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city, pop in zip(df2['Latitude'], df2['Longitude'], df2['City'], df2['pop_april_2010']):
    label = '{}, {}'.format(city, pop)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map1)  
    
map1

In [261]:
# define function for getting venues from Foursquare API

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    LIMIT=500
    venues_list=[]
        
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

    return(nearby_venues)

In [262]:
# getting venues of San Diego from Foursquare API
sd_venues = getNearbyVenues(names=df2['City'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )

Carlsbad
Chula Vista
Coronado
Del Mar
El Cajon
Encinitas
Escondido
Imperial Beach
La Mesa
Lemon Grove
National City
Oceanside
Poway
San Diego
San Marcos
Santee
Solana Beach
Vista


In [263]:
# check venue result
print(sd_venues.shape)
sd_venues.head()
sd_venues.tail()

(699, 7)


,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
694,Vista,33.200036,-117.242536,Vista Village Plaza,33.200770,-117.245609,Shopping Mall
695,Vista,33.200036,-117.242536,Vista Transit Center,33.202995,-117.244811,Light Rail Station
696,Vista,33.200036,-117.242536,Crunch Fitness,33.197572,-117.245898,Gym / Fitness Center
697,Vista,33.200036,-117.242536,Rodeo's Market,33.200669,-117.246968,Grocery Store
698,Vista,33.200036,-117.242536,Rusty Nail Renovations,33.195812,-117.240913,Moving Target


In [264]:
# filter the grocery stores from all the cities in San Diego
df4=pd.DataFrame()
df4=sd_venues[sd_venues['Venue Category']=='Grocery Store']
df4

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
257,Escondido,33.119208,-117.086422,Smart & Final Extra!,33.122506,-117.087514,Grocery Store
280,Imperial Beach,32.583944,-117.113086,Grocery Outlet,32.583659,-117.114633,Grocery Store
299,La Mesa,32.767828,-117.023083,VONS,32.765336,-117.024285,Grocery Store
341,Lemon Grove,32.742553,-117.031417,Sprouts Farmers Market,32.740505,-117.030014,Grocery Store
401,National City,32.678108,-117.099197,Chan Dong Oriental Supermarket,32.677389,-117.100612,Grocery Store
402,National City,32.678108,-117.099197,Mothers nutritional center,32.676662,-117.098464,Grocery Store
567,San Diego,32.715328,-117.157256,Grocery Outlet,32.711764,-117.155108,Grocery Store
586,San Marcos,33.143372,-117.166144,San Marcos Gym,33.142503,-117.166866,Grocery Store
663,Vista,33.200036,-117.242536,Frazier Farms,33.197798,-117.246055,Grocery Store
697,Vista,33.200036,-117.242536,Rodeo's Market,33.200669,-117.246968,Grocery Store


In [265]:
# count the grocery stores by cities
df5=df4.groupby('City')['Venue Category'].count()
df5=pd.DataFrame(df5).reset_index()
df5

,City,Venue Category
0,Escondido,1
1,Imperial Beach,1
2,La Mesa,1
3,Lemon Grove,1
4,National City,2
5,San Diego,1
6,San Marcos,1
7,Vista,2


In [281]:
# merge city basic info and numbers of grocery stores in each city
df6=pd.merge(df3, df5, how='outer')
df6.rename(columns={'Venue Category':'Grocery Store'}, inplace=True)
df6.replace(np.nan,'0', inplace=True)
df6

,City,pop_april_2010,Latitude,Longitude,Grocery Store
0,Carlsbad,105328,33.158092,-117.350594,0
1,Chula Vista,243916,32.640053,-117.084197,0
2,Coronado,18912,32.685886,-117.183089,0
3,Del Mar,4161,32.959489,-117.265314,0
4,El Cajon,99478,32.794772,-116.962528,0
5,Encinitas,59518,33.036986,-117.291983,0
6,Escondido,143911,33.119208,-117.086422,1
7,Imperial Beach,26324,32.583944,-117.113086,1
8,La Mesa,57065,32.767828,-117.023083,1
9,Lemon Grove,25320,32.742553,-117.031417,1


In [282]:
# new store share: how many people share a grocery store if a new store opens in each city
# (number of grocery stores + 1) divided by population of each city
df6['pop_april_2010']=df6['pop_april_2010'].astype(int)
df6['Grocery Store']=df6['Grocery Store'].astype(int)
df6['New Store Share']= df6['pop_april_2010']/(df6['Grocery Store']+1)
df6['New Store Share']=df6['New Store Share'].astype(int)
df6

,City,pop_april_2010,Latitude,Longitude,Grocery Store,New Store Share
0,Carlsbad,105328,33.158092,-117.350594,0,105328
1,Chula Vista,243916,32.640053,-117.084197,0,243916
2,Coronado,18912,32.685886,-117.183089,0,18912
3,Del Mar,4161,32.959489,-117.265314,0,4161
4,El Cajon,99478,32.794772,-116.962528,0,99478
5,Encinitas,59518,33.036986,-117.291983,0,59518
6,Escondido,143911,33.119208,-117.086422,1,71955
7,Imperial Beach,26324,32.583944,-117.113086,1,13162
8,La Mesa,57065,32.767828,-117.023083,1,28532
9,Lemon Grove,25320,32.742553,-117.031417,1,12660


In [285]:
# sort data by population and number of grocery stores
df7=df6.sort_values(by=['New Store Share'], ascending=False)
df7 = df7.reset_index(drop=True)
df7

,City,pop_april_2010,Latitude,Longitude,Grocery Store,New Store Share
0,San Diego,1307402,32.715328,-117.157256,1,653701
1,Chula Vista,243916,32.640053,-117.084197,0,243916
2,Oceanside,167086,33.195869,-117.379483,0,167086
3,Carlsbad,105328,33.158092,-117.350594,0,105328
4,El Cajon,99478,32.794772,-116.962528,0,99478
5,Escondido,143911,33.119208,-117.086422,1,71955
6,Encinitas,59518,33.036986,-117.291983,0,59518
7,Santee,53413,32.838383,-116.973917,0,53413
8,Poway,47811,32.962822,-117.035864,0,47811
9,San Marcos,83781,33.143372,-117.166144,1,41890


In [293]:
# the city of most people share one grocery store
print(df7.iloc[0,0], 'is the new grocery store location')

San Diego is the new grocery store location


In [296]:
# filter all the venues in the city of 'San Diego'
df8=pd.DataFrame()
df8=sd_venues[sd_venues['City']=='San Diego']
print(df8.shape)
df8.head()

(100, 7)


,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
473,San Diego,32.715328,-117.157256,Best Damn Beer Shop,32.716203,-117.158550,Beer Store
474,San Diego,32.715328,-117.157256,Art Hookah Lounge,32.714935,-117.159274,Lounge
475,San Diego,32.715328,-117.157256,Chef Miguel's Deli,32.716500,-117.158690,Sandwich Place
476,San Diego,32.715328,-117.157256,Dog Haus Biergarten San Diego,32.715529,-117.156307,Hot Dog Joint
477,San Diego,32.715328,-117.157256,Pokez Mexican Restaurant,32.714574,-117.155761,Mexican Restaurant


In [312]:
# unique venue category
# a=df8['Venue Category'].unique()
# a
#a.count()

In [313]:
# filter all the restaurant
df9=df8[df8['Venue Category'].str.contains('Restaurant')]
print(df9.shape)
df9.head()

(34, 7)


,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
477,San Diego,32.715328,-117.157256,Pokez Mexican Restaurant,32.714574,-117.155761,Mexican Restaurant
480,San Diego,32.715328,-117.157256,Tajima Ramen East Village,32.714553,-117.156398,Ramen Restaurant
486,San Diego,32.715328,-117.157256,The Crab Hut,32.715802,-117.159871,Seafood Restaurant
490,San Diego,32.715328,-117.157256,The Melting Pot,32.714844,-117.159930,Fondue Restaurant
491,San Diego,32.715328,-117.157256,Empanada Kitchen,32.716635,-117.157114,Empanada Restaurant


In [314]:
# map all the restaurant in the city of San Diego using latitude and longitude values
latitude = 32.71528
longitude = -117.157256
#print('The geograpical coordinate of San Diego are {}, {}.'.format(latitude, longitude))

map2 = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, venue, cat in zip(df9['Venue Latitude'], df9['Venue Longitude'], df9['Venue'], df9['Venue Category']):
    label = '{}, {}'.format(venue, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map2)  
    
map2

In [345]:
# DBSCAN clustering based on their location i.e. Lat & Lon
from sklearn.cluster import DBSCAN
import sklearn.utils
from sklearn.preprocessing import StandardScaler
sklearn.utils.check_random_state(1000)
Clus_dataSet = df9[['Venue Latitude','Venue Longitude']]
Clus_dataSet = np.nan_to_num(Clus_dataSet)
Clus_dataSet = StandardScaler().fit_transform(Clus_dataSet)

# Compute DBSCAN
db = DBSCAN(eps=0.00003, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
df9["Clus_Db"]=labels

realClusterNum=len(set(labels)) - (1 if -1 in labels else 0)
clusterNum = len(set(labels)) 

# A sample of clusters
df10=df9[["Venue","Venue Latitude","Venue Longitude","Clus_Db"]]
df10

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Venue,Venue Latitude,Venue Longitude,Clus_Db
477,Pokez Mexican Restaurant,32.714574,-117.155761,0
480,Tajima Ramen East Village,32.714553,-117.156398,0
486,The Crab Hut,32.715802,-117.159871,1
490,The Melting Pot,32.714844,-117.159930,1
491,Empanada Kitchen,32.716635,-117.157114,1
492,Brian's 24,32.714084,-117.159313,1
494,Time Out Sports Tavern,32.715840,-117.158586,1
498,Osetra the Fishhouse,32.714777,-117.160314,1
500,Trailer Park After Dark,32.714195,-117.160076,1
502,Buca di Beppo,32.712895,-117.159095,1


In [348]:
# visualize the resulting clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=16)

# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df10['Venue Latitude'], df10['Venue Longitude'], df10['Venue'], df10['Clus_Db']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [359]:
# find the centroid for each cluster
from sklearn.neighbors.nearest_centroid import NearestCentroid
import numpy as np
X = np.array(df10[['Venue Latitude','Venue Longitude']])
y = np.array(df10['Clus_Db'])
clf = NearestCentroid()
clf.fit(X, y)

print(clf.centroids_)

[[  32.71456327 -117.15607955]
 [  32.71447377 -117.15973498]
 [  32.71675075 -117.1539721 ]
 [  32.71159854 -117.15630906]]


In [361]:
# cluster 1 has the most restaurant nearby, its centroid would be the best place for the grocery store.
print('The best location for the new grocery store is', clf.centroids_[0])

The best location for the new grocery store is [  32.71456327 -117.15607955]
